# Automatic Differentiation with torch.autograd

In [1]:
%matplotlib inline

In [2]:
import torch

x = torch.ones(5) #input tensor
y = torch.zeros(3) #expected output(正解ラベル)
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [3]:
#grad_fnはそのテンソルがどの演算結果として生まれたかを表す情報
print('Gradient function for z =', z.grad_fn) #zは加算(Add)演算で得られた
                                              #計算グラフのノード名が AddBackward0
print('Gradient function for loss =', loss.grad_fn) #lossはBCE-with-logits演算の結果
                                                    #計算グラフのノード名がBinaryCrossEntropyWithLogitsBackward

Gradient function for z = <AddBackward0 object at 0x7158a8a61db0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7158a8a617b0>


In [4]:
#gradにはそのテンソル（通常は学習パラメータ）に関する損失の勾配値が格納される
loss.backward()
print(w.grad)  # wに対する損失の勾配
print(b.grad)  # bに対する損失の勾配

tensor([[0.3144, 0.0742, 0.0005],
        [0.3144, 0.0742, 0.0005],
        [0.3144, 0.0742, 0.0005],
        [0.3144, 0.0742, 0.0005],
        [0.3144, 0.0742, 0.0005]])
tensor([0.3144, 0.0742, 0.0005])


In [5]:
#勾配計算をしない方法(訓練済みモデルで推論するケースなど)
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


## 1. コンテキストマネージャーとは

**「ある処理の前後に、決まった準備と後片付けを自動でやってくれる仕組み」**
Pythonでは、この仕組みを使うオブジェクトを **コンテキストマネージャー** と呼びます。

コンテキストマネージャーは、次の2つのメソッドを持っている必要があります：

* `__enter__(self)`
  → `with` ブロックに入るときに実行される処理（初期化・準備）
* `__exit__(self, exc_type, exc_val, exc_tb)`
  → `with` ブロックを抜けるときに必ず実行される処理（後片付け）
  ※ブロック内で例外が発生しても実行される

---

## 2. `with` 文とコンテキストマネージャーの関係

`with` 文は、コンテキストマネージャーを呼び出して、その**前後処理**を自動で挟んでくれます。

### 流れ

```python
with コンテキスト式 as 変数:
    ブロック処理
```

実際の動作イメージ：

1. `コンテキスト式.__enter__()` が呼ばれる
   → 返り値が `変数` に代入される
2. ブロックの処理が実行される
3. ブロック終了時に `コンテキスト式.__exit__()` が呼ばれる
   → 例外があっても必ず呼ばれる

---

## 3. ファイル操作の例 (`with open(...) as f`)

```python
with open("sample.txt", "r") as f:
    data = f.read()
    print(data)
```

* `open()` が返すファイルオブジェクトはコンテキストマネージャー
* `f.__enter__()` → ファイルを開く（ハンドルを返す）
* `f.__exit__()` → ファイルを閉じる（後片付け）

### なぜ便利か？

* `with` を使わない場合

```python
f = open("sample.txt", "r")
try:
    data = f.read()
finally:
    f.close()  # 例外が出ても必ず閉じるようにする
```

* `with` を使えばこれを1行で安全に書ける

---

## 4. `torch.no_grad()` もコンテキストマネージャー

```python
with torch.no_grad():
    # この中はautogradが無効になる
```

* `__enter__()` → autogradを一時的にOFF
* `__exit__()` → 元のautograd設定に戻す

つまり「勾配追跡のオン/オフ」を**安全に自動で切り替えるためのコンテキストマネージャー**です。

---

## 5. まとめ

* **コンテキストマネージャー** = 前後処理を自動化するオブジェクト
* **`with` 文** = コンテキストマネージャーを使う構文
* ファイル操作、勾配無効化、ロック取得・解放、DB接続管理などに広く使われる


In [6]:
#detach() は PyTorch のテンソルメソッドで、そのテンソルを計算グラフから切り離した新しいテンソルを返すためのもの
z = torch.matmul(x, w)+b
z_det = z.detach() #元のテンソルと同じ値を持つが、autograd（自動微分）の追跡対象外になる
print(z_det.requires_grad)

False
